# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
from sqlalchemy import create_engine, select
import sqlite3

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords','omw-1.4'])

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Database.db')
df =  pd.read_sql(
    "SELECT * FROM messagesdf",
    con=engine )
X = df[['id','message' ,'original' ,'genre']]
Y = df.drop(['id','message' ,'original' ,'genre'], axis =1 )

### 2. Write a tokenization function to process your text data

In [3]:
#url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
#url_regex = '//(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))'
swords = stopwords.words('english')
def tokenize(text):


    #detected_urls = re.findall(url_regex, text)
    #for url in detected_urls:
        #text = text.replace(url, "urlplaceholder")



    lista = ["[^a-zA-Z]" , "http\S+" ,"http" ,"@\S+" ,  "[^A-Za-z0-9(),!?@\'\`\"\_\n]" , "@" ]
    for item in lista:
        text = re.sub(item, " ", text)



    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    tokens = [tok.lower().strip() for tok in tokens if tok.lower() not in swords]

    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens ]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
vect = CountVectorizer(tokenizer=tokenize, min_df=100, analyzer='word')
tfidf = TfidfTransformer(smooth_idf=False)
clf =GaussianNB()

In [5]:
test = vect.fit_transform(X['message'])

In [6]:
test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
vect.stop_words_

{'oelhouston',
 'cheaper',
 'aridity',
 'uncomplicated',
 'nkhosi',
 'evaluator',
 'knee',
 'brt',
 'deposition',
 'remembered',
 'mind',
 'sometents',
 'storing',
 'powerless',
 'pastoralism',
 'bipartisan',
 'kibondo',
 'verdean',
 'carte',
 'suddenly',
 'lxwuovery',
 'utilising',
 'utero',
 'wap',
 'pronlongue',
 'quilt',
 'uzlkwcyy',
 'mariani',
 'biting',
 'grover',
 'pulpers',
 'reverberation',
 'sokuluk',
 'ancestral',
 'federer',
 'sweeterynyc',
 'bxnpppgs',
 'complementary',
 'amerique',
 'pill',
 'boko',
 'mayard',
 'clinic',
 'neighbour',
 'benguela',
 'retention',
 'sandra',
 'kala',
 'sheenimit',
 'sahel',
 'elisa',
 'tryna',
 'overhaul',
 'insanity',
 'distillation',
 'flop',
 'mous',
 'aznar',
 'eventual',
 'unable',
 'stil',
 'expects',
 'whipping',
 'upazila',
 'aldhanhani',
 'sagar',
 'chokwe',
 'vellage',
 'suckitsandy',
 'craftsman',
 'baruch',
 'jdq',
 'hayn',
 'ittihad',
 'instrumental',
 'whilst',
 'cuppa',
 'constituent',
 'intra',
 'humankind',
 'acceptible',
 

In [8]:
pd.DataFrame( data =  test.toarray() , columns=vect.vocabulary_).sum().sort_values(ascending=False ).head(20)

crop              3037
year              3006
well              2898
enough            2652
far               2490
hand              2045
essential         1926
tomorrow          1918
area              1664
make              1527
rehabilitation    1489
reconstruction    1350
found             1251
would             1114
committee         1113
done              1056
political         1042
helicopter        1020
among             1009
available          991
dtype: int64

In [9]:
len(vect.vocabulary_)

686

In [10]:
test2 = tfidf.fit_transform(test)

In [11]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [12]:
Y['hospitals'].value_counts()

0    25897
1      283
Name: hospitals, dtype: int64

In [13]:
list(vect.get_feature_names_out())

['able',
 'access',
 'according',
 'aceh',
 'across',
 'action',
 'activity',
 'acute',
 'added',
 'addition',
 'additional',
 'address',
 'affected',
 'afghanistan',
 'afp',
 'africa',
 'african',
 'aftershock',
 'agency',
 'agricultural',
 'agriculture',
 'aid',
 'air',
 'airport',
 'almost',
 'along',
 'already',
 'also',
 'american',
 'among',
 'animal',
 'another',
 'answer',
 'anything',
 'april',
 'area',
 'armed',
 'army',
 'around',
 'arrived',
 'ask',
 'asking',
 'assessment',
 'assist',
 'assistance',
 'attack',
 'au',
 'august',
 'authority',
 'available',
 'away',
 'b',
 'baby',
 'back',
 'bad',
 'bangladesh',
 'bank',
 'based',
 'basic',
 'become',
 'better',
 'big',
 'bit',
 'blanket',
 'boat',
 'body',
 'border',
 'bridge',
 'bring',
 'build',
 'building',
 'built',
 'c',
 'call',
 'called',
 'came',
 'camp',
 'campaign',
 'capacity',
 'capital',
 'car',
 'card',
 'care',
 'carrefour',
 'carrying',
 'case',
 'cause',
 'caused',
 'causing',
 'center',
 'central',
 'centr

In [16]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26177,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26178,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
clf.fit(test2.toarray() , Y['related'])

GaussianNB()

In [18]:
clf.score(test2.toarray() , Y['related'])


0.7118792971734148

In [53]:
Y = Y.loc[:,Y.sum()>0]

In [42]:
estimators = [('vect' , CountVectorizer(tokenizer=tokenize, min_df=300)) , \
              ('tfidf' ,TfidfTransformer(smooth_idf=False) ) , \
              ('clf',MultiOutputClassifier(RandomForestClassifier(max_depth=50, class_weight='balanced')))]
pipeline = Pipeline(estimators)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [43]:
x_train, x_test, y_train , y_test = train_test_split(X, Y , test_size=0.2, random_state=1)

In [44]:
pipeline.fit(x_train['message'], y_train.iloc[:,0:4])

Pipeline(steps=[('vect',
                 CountVectorizer(min_df=300,
                                 tokenizer=<function tokenize at 0x00000249F16FE440>)),
                ('tfidf', TfidfTransformer(smooth_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        max_depth=50)))])

In [45]:
pipeline.score(x_train['message'], y_train.iloc[:,0:4])

0.7101795263559969

In [46]:
pipeline.score(x_test['message'], y_test.iloc[:,0:4])

0.47612681436210846

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.